# Fundamental Agent 基本面分析測試

本測試展示：**基本面分析 Agent 的完整工作流程**

使用 5 個自訂基本面分析工具（@function_tool）進行手動測試：
- calculate_financial_ratios
- analyze_financial_health
- evaluate_valuation
- analyze_growth_potential
- generate_investment_rating

## Step 1: 環境設置

In [1]:
import sys
from pathlib import Path
from dotenv import load_dotenv

from agents import (
    Runner,
    gen_trace_id,
    trace,
    WebSearchTool,
    CodeInterpreterTool,
)
from agents.mcp import MCPServerStdio

load_dotenv()

project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root / 'backend' / 'src'))

from common.logger import logger

print('✓ 環境設置完成')

2025-10-19 18:13:01 | INFO     | common.logger:setup_logger:100 | Logger initialized with level: INFO
2025-10-19 18:13:01 | INFO     | common.logger:setup_logger:102 | Logging to file: /Users/sacahan/Documents/workspace/CasualTrader/backend/logs/casualtrader.log


✓ 環境設置完成


## Step 2: 導入必要的模組

In [2]:
from trading.tools.fundamental_agent import (
    get_fundamental_agent,
)

print('✓ 模組導入完成')

✓ 模組導入完成


## Step 3: 初始化 OpenAI 內建工具

In [3]:
# WebSearchTool: 提供網路搜尋功能
web_search_tool = WebSearchTool(
    user_location=None,  # 可選：用戶位置，用於本地化搜尋結果
    filters=None,  # 可選：搜尋過濾器
    search_context_size="medium",  # 搜尋上下文大小：'low'、'medium'、'high'
)

# CodeInterpreterTool: 提供程式碼執行功能
# 必須指定 type 和 container 設置，container.type 必須為 "auto"
code_interpreter_tool = CodeInterpreterTool(
    tool_config={
        "type": "code_interpreter",
        "container": {
            "type": "auto"  # OpenAI 自動選擇最適合的容器
        },
    }
)

openai_tools = [web_search_tool, code_interpreter_tool]

## Step 4: 初始化 MCP Servers

In [4]:
# 全局變量來管理 MCP servers
casual_market_mcp = None
mcp_memory = None
mcp_servers = []

async def setup_mcp_servers():
    """設置 MCP servers"""
    global casual_market_mcp, mcp_memory, mcp_servers
    
    # 清理舊的連接
    if casual_market_mcp:
        try:
            await casual_market_mcp.__aexit__(None, None, None)
        except:
            pass
    
    if mcp_memory:
        try:
            await mcp_memory.__aexit__(None, None, None)
        except:
            pass
    
    # 初始化 casual-market-mcp
    casual_market_mcp = MCPServerStdio(
        params={
            "command": "uvx",
            "args": [
                "--from",
                "/Users/sacahan/Documents/workspace/CasualMarket",
                "casual-market-mcp",
            ],
        }, 
        name="casual_market_mcp", 
        client_session_timeout_seconds=120  # 增加超時時間
    )
    
    # 初始化 mcp-memory-libsql
    mcp_memory = MCPServerStdio(
        params={
            "command": "npx",
            "args": ["-y", "mcp-memory-libsql"],
            "env": {"LIBSQL_URL": "file:./labs/fundamental_agent.db"},
        },
        name="mcp_memory",
        client_session_timeout_seconds=120,  # 增加超時時間
    )
    
    # 連接 servers
    await casual_market_mcp.__aenter__()
    await mcp_memory.__aenter__()
    
    mcp_servers = [casual_market_mcp, mcp_memory]
    print('✓ MCP Servers 已連接')

await setup_mcp_servers()

✓ MCP Servers 已連接


## Step 5: 初始化 Fundamental Agent

In [5]:
# 創建基本面分析 Agent
fundamental_agent = await get_fundamental_agent(
    model_name="gpt-4o-mini",
    mcp_servers=mcp_servers,
    openai_tools=openai_tools,
    max_turns=15,
)

print('✓ Fundamental Agent 已初始化')

2025-10-19 18:13:50 | INFO     | trading.tools.fundamental_agent:get_fundamental_agent:583 | get_fundamental_agent() called with model=gpt-4o-mini
2025-10-19 18:13:50 | INFO     | trading.tools.fundamental_agent:get_fundamental_agent:598 | Creating Agent with model=gpt-4o-mini, mcp_servers=2, tools=7
2025-10-19 18:13:50 | INFO     | trading.tools.fundamental_agent:get_fundamental_agent:613 | Fundamental Analyst Agent created successfully


✓ Fundamental Agent 已初始化


## 測試 1: 單一公司基本面分析

In [ ]:
async def test_single_company_analysis():
    print('\n' + '='*70)
    print('測試 1: 單一公司基本面分析 - 台積電 (2330)')
    print('='*70)
    
    prompt = '''請分析台積電 (2330) 的基本面。
    
最新財務數據：
- 營收：新台幣 250 億元
- 淨利：新台幣 50 億元
- 總資產：新台幣 1,200 億元
- 股東權益：新台幣 600 億元
- 市值：新台幣 10,000 億元
- 目前股價：新台幣 450 元

請進行以下分析：
1. 計算主要財務比率 (ROE, ROA, 淨利率)
2. 評估財務體質健全度
3. 評估當前估值水準
4. 分析成長潛力
5. 最後給出投資評級和建議'''
    
    print(f'\n用戶提示:\n{prompt}')
    print('\n→ Agent 執行分析...')
    print('-' * 70)
    
    trace_id = gen_trace_id()
    try:
        with trace(workflow_name="fundamental_agent_lab", trace_id=trace_id):
            result = await Runner.run(fundamental_agent, prompt, max_turns=15)
    
        print('-' * 70)
        print('\nAgent 分析結果:')
        print(result.final_output)
        
        return result
    except Exception as e:
        print(f"執行錯誤: {e}")
        print(f"錯誤類型: {type(e).__name__}")
        return None

result_1 = await test_single_company_analysis()

## 測試 2: 完整 Agent 工作流程 - 多公司對比

In [6]:
async def test_multi_company_comparison():
    print('\n' + '='*70)
    print('測試 2: 完整 Agent 工作流程 - 多公司對比分析')
    print('='*70)
    
    prompt = '''比較以下三家科技公司的基本面：
    
公司 A (2330)：
- ROE: 25%
- PE 比率: 20 倍
- 淨利潤率: 18%
- 營收成長: 15%
- EPS 成長: 18%

公司 B (2454)：
- ROE: 18%
- PE 比率: 25 倍
- 淨利潤率: 14%
- 營收成長: 22%
- EPS 成長: 28%

公司 C (3680)：
- ROE: 15%
- PE 比率: 35 倍
- 淨利潤率: 10%
- 營收成長: 8%
- EPS 成長: 5%

請分析：
1. 各公司的優劣勢
2. 估值的相對吸引力
3. 成長潛力排名
4. 投資建議排序
5. 風險評估'''
    
    print(f'\n用戶提示:\n{prompt}')
    print('\n→ Agent 執行分析...')
    print('-' * 70)
    
    trace_id = gen_trace_id()
    try:
        with trace(workflow_name="fundamental_agent_lab", trace_id=trace_id):
            result = await Runner.run(fundamental_agent, prompt, max_turns=15)
    
        print('-' * 70)
        print('\nAgent 分析結果:')
        print(result.final_output)
        
        return result
    except Exception as e:
        print(f"執行錯誤: {e}")
        print(f"錯誤類型: {type(e).__name__}")
        return None

result_2 = await test_multi_company_comparison()


測試 2: 完整 Agent 工作流程 - 多公司對比分析

用戶提示:
比較以下三家科技公司的基本面：

公司 A (2330)：
- ROE: 25%
- PE 比率: 20 倍
- 淨利潤率: 18%
- 營收成長: 15%
- EPS 成長: 18%

公司 B (2454)：
- ROE: 18%
- PE 比率: 25 倍
- 淨利潤率: 14%
- 營收成長: 22%
- EPS 成長: 28%

公司 C (3680)：
- ROE: 15%
- PE 比率: 35 倍
- 淨利潤率: 10%
- 營收成長: 8%
- EPS 成長: 5%

請分析：
1. 各公司的優劣勢
2. 估值的相對吸引力
3. 成長潛力排名
4. 投資建議排序
5. 風險評估

→ Agent 執行分析...
----------------------------------------------------------------------


2025-10-19 18:14:03 | INFO     | httpx._client:_send_single_request:1740 | HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-10-19 18:14:04 | INFO     | trading.tools.fundamental_agent:calculate_financial_ratios:228 | 開始計算財務比率 | 股票: 2330
2025-10-19 18:14:04 | INFO     | trading.tools.fundamental_agent:calculate_financial_ratios:274 | 財務比率計算完成 | 股票: 2330
2025-10-19 18:14:05 | INFO     | httpx._client:_send_single_request:1025 | HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"
2025-10-19 18:14:10 | INFO     | httpx._client:_send_single_request:1740 | HTTP Request: POST https://api.openai.com/v1/responses "HTTP/1.1 200 OK"
2025-10-19 18:14:11 | INFO     | trading.tools.fundamental_agent:calculate_financial_ratios:228 | 開始計算財務比率 | 股票: 2454
2025-10-19 18:14:11 | INFO     | trading.tools.fundamental_agent:calculate_financial_ratios:274 | 財務比率計算完成 | 股票: 2454
2025-10-19 18:14:14 | INFO     | httpx._client:_send_single_request:1740

----------------------------------------------------------------------

Agent 分析結果:
### 1. 各公司的優劣勢分析

#### 公司 A (2330)
**優勢:**
- **高 ROE (25%)**：顯示公司在股東權益上的獲利能力強。
- **合理 PE 比率 (20 倍)**：在同行業中，相對較低的 P/E 比率，顯示估值合理。

**劣勢:**
- 營收成長率 (15%) 和 EPS 成長 (18%) 雖然不低，但相對於競爭對手不算特出。

#### 公司 B (2454)
**優勢:**
- **高營收成長 (22%) 和 EPS 成長 (28%)**：顯示未來潛力強勁，市場需求良好。
- 雖然 ROE (18%) 和淨利潤率 (14%) 比較低，但仍在行業中表現良好。

**劣勢:**
- **高 PE 比率 (25 倍)**：顯示市場對其未來增長前景的高期待，也會帶來較高的估值風險。

#### 公司 C (3680)
**優勢:**
- 營收成長和 EPS 成長穩定，提供可靠的收益來源。

**劣勢:**
- **低 ROE (15%) 和淨利潤率 (10%)**：顯示獲利能力較弱。
- **高 PE 比率 (35 倍)**：顯示投資者對其未來展望的期望過高，相對風險增加。

---

### 2. 估值的相對吸引力
- **公司 A (2330)**：PE 比率 (20 倍) 是三家公司中最低，顯示評價相對便宜。
- **公司 B (2454)**：PE 比率 (25 倍)，比公司 A 更高，但若他們的增長實現，仍可接受。
- **公司 C (3680)**：PE 比率 (35 倍) 明顯過高，需謹慎。

### 3. 成長潛力排名
1. **公司 B (2454)**：最高的營收和 EPS 成長率，潛力大。
2. **公司 A (2330)**：穩定的成長，雖然增長較慢但仍有可觀的未來。
3. **公司 C (3680)**：最低的成長率，潛力有限。

### 4. 投資建議排序
1. **公司 B (2454)**：看好其成長潛力，強烈建議買入。
2. **公司 A (2330)**：在合理價位及穩定獲利，建議持有。
3. **公司 C (3680)**：估值

2025-10-19 18:14:31 | INFO     | httpx._client:_send_single_request:1025 | HTTP Request: POST https://api.openai.com/v1/traces/ingest "HTTP/1.1 204 No Content"


## 測試 3: 完整 Agent 工作流程 - 綜合基本面評估

In [ ]:
async def test_comprehensive_fundamental_analysis():
    print('\n' + '='*70)
    print('測試 3: 完整 Agent 工作流程 - 綜合基本面評估')
    print('='*70)
    
    prompt = '''進行完整的公司基本面綜合評估。
    
公司背景：
- 產業：半導體
- 市場地位：全球領導者
- 營運歷史：20 年以上

財務狀況：
- 三年平均 ROE: 22%
- 淨債務比：0.2
- 現金流狀況：強勁
- 股利政策：持續分配

市場評價：
- 當前 PE 比率：18 倍
- 產業平均 PE：20 倍
- 過去 1 年股價表現：+30%
- 技術分析：上升趨勢

發展前景：
- AI 芯片需求激增
- 產能擴充計畫
- 海外市場拓展

請進行全面的基本面評估，結合所有因素給出綜合投資評級和長期持有建議。'''
    
    print(f'\n用戶提示:\n{prompt}')
    print('\n→ Agent 執行綜合分析...')
    print('-' * 70)
    
    trace_id = gen_trace_id()
    try:
        with trace(workflow_name="fundamental_agent_lab", trace_id=trace_id):
            result = await Runner.run(fundamental_agent, prompt, max_turns=15)
    
        print('-' * 70)
        print('\nAgent 分析結果:')
        print(result.final_output)
        
        return result
    except Exception as e:
        print(f"執行錯誤: {e}")
        print(f"錯誤類型: {type(e).__name__}")
        return None
    
    return result

result_3 = await test_comprehensive_fundamental_analysis()

## 清理資源

In [ ]:
async def cleanup_mcp_servers():
    """清理 MCP servers 連接"""
    global casual_market_mcp, mcp_memory
    
    try:
        if casual_market_mcp:
            await casual_market_mcp.__aexit__(None, None, None)
            print('✓ casual_market_mcp 已關閉')
        
        if mcp_memory:
            await mcp_memory.__aexit__(None, None, None)
            print('✓ mcp_memory 已關閉')
            
        print('✓ 所有 MCP servers 已清理完成')
        
    except Exception as e:
        print(f'清理過程中發生錯誤: {e}')

# 測試完成後執行清理
await cleanup_mcp_servers()